# Researching best params for new approach
On this notebook, we try to find the best params for our suggested approach on ICQ classifier.

## Setup

### Importing libraries and helper methods

In [1]:
import sys, os
sys.path.append(os.path.abspath('../../../classifiers'))
sys.path.append(os.path.abspath('../../../training'))
sys.path.append(os.path.abspath('../../../validation'))

In [2]:
import warnings
from sklearn.exceptions import UndefinedMetricWarning

# We're ignoring some warning from sklearn.metrics.classification_report
warnings.simplefilter(action='ignore', category=UndefinedMetricWarning)

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [3]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

### Setting up database, k-fold and random seed

In [4]:
seed = 40

# This decides whether we're trying our proposed approach (True) or the original one (False)
load_inputvector_env_state = False

# This decides whether we normalize per feature (0) or per instance (1)
normalize_axis = 0

In [5]:
iris = datasets.load_iris()

X = iris.data[:, [0,1,2,3]]
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
cv = StratifiedKFold(n_splits=10, random_state=seed, shuffle=True)

## Search for best params on sigmaQ = sigmaX + sigmaY + sigmaZ

### Model

In [6]:
from iqc_classifier_training import IQCClassifier
from sklearn.multiclass import OneVsRestClassifier
from classifiers_ICQ import iqc_classifier

In [7]:
dic_training_params = {"max_iter": 1000,
    "accuracy_succ": 0.99,
    "plot_graphs_and_metrics": False,
    "plot_graphs_in_classifier": False,
    "random_seed": 1,
    "learning_rate": 0.01,
    "do_classes_refit":True,
    "reset_weights_epoch":0,
    "batch":1}

In [8]:
dic_classifier_params = {}
dic_classifier_params["sigma_q_params"] = [1,1,1,0]
dic_classifier_params["use_polar_coordinates_on_sigma_q"] = False
dic_classifier_params["load_inputvector_env_state"] = load_inputvector_env_state
dic_classifier_params["normalize_axis"] = normalize_axis

In [9]:
icq = OneVsRestClassifier(IQCClassifier(
                        classifier_function=iqc_classifier, 
                        dic_classifier_params=dic_classifier_params,
                        dic_training_params=dic_training_params), n_jobs=-1, verbose=1)

### Search for Sigma Q Params
First research on the new approach (having weights on U operator and inputs on rho env) - varying SigmaQ params

In [10]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

#### Integer params
On this phase, we're dealing only with integer params, i.e. [1, 1, 1, 1], [2, 2, 2, 2], etc.

In [11]:
params = dict()
params["estimator__dic_training_params"] = [dic_training_params]
params["estimator__classifier_function"] = [iqc_classifier]
params["estimator__dic_classifier_params"] = []
for i in range(0, 15, 1):
    for j in range(0, 15, 1):
        for k in range(0, 15, 1):
            for l in range(0, 15, 1):
                    dic_classifier_params = {}
                    dic_classifier_params["sigma_q_params"] = [i, j, k, l]
                    dic_classifier_params["use_polar_coordinates_on_sigma_q"] = False
                    dic_classifier_params["load_inputvector_env_state"] = load_inputvector_env_state
                    dic_classifier_params["normalize_axis"] = normalize_axis
                    params["estimator__dic_classifier_params"].append(dic_classifier_params)

In [12]:
%%time
busca = RandomizedSearchCV(icq, params, n_iter=100, scoring='accuracy', n_jobs=-1, cv=cv, random_state=seed)

# Executa busca
resultado = busca.fit(X_train, y_train)

# Resume resultados
print('Resultados busca - IQC Alterado')
print('Melhor acurácia: %s' % resultado.best_score_)
print('Melhor hiperparâmetro: %s' % resultado.best_params_)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Resultados busca - IQC Alterado
Melhor acurácia: 0.875
Melhor hiperparâmetro: {'estimator__dic_training_params': {'max_iter': 1000, 'accuracy_succ': 0.99, 'plot_graphs_and_metrics': False, 'plot_graphs_in_classifier': False, 'random_seed': 1, 'learning_rate': 0.01, 'do_classes_refit': True, 'reset_weights_epoch': 0, 'batch': 1, 'coupling_constants': [1]}, 'estimator__dic_classifier_params': {'sigma_q_params': [1, 6, 5, 13], 'use_polar_coordinates_on_sigma_q': False, 'load_inputvector_env_state': False, 'normalize_axis': 0}, 'estimator__classifier_function': <function iqc_classifier at 0x000002123EE24EE0>}
CPU times: total: 2.2 s
Wall time: 10h 38min 4s


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  1.3min finished


#### Float params
On this phase, we'll deal with Sigma Q params as floats [0,1].

In [13]:
import numpy as np

In [14]:
params = dict()
params["estimator__dic_training_params"] = [dic_training_params]
params["estimator__classifier_function"] = [iqc_classifier]
params["estimator__dic_classifier_params"] = []
for i in np.arange(0, 1, 0.1):
    for j in np.arange(0, 1, 0.1):
        for k in np.arange(0, 1, 0.1):
            for l in np.arange(0, 1, 0.1):
                    dic_classifier_params = {}
                    dic_classifier_params["sigma_q_params"] = [i, j, k, l]
                    dic_classifier_params["use_polar_coordinates_on_sigma_q"] = False
                    dic_classifier_params["load_inputvector_env_state"] = load_inputvector_env_state
                    dic_classifier_params["normalize_axis"] = 0
                    params["estimator__dic_classifier_params"].append(dic_classifier_params)

In [15]:
%%time
busca = RandomizedSearchCV(icq, params, n_iter=100, scoring='accuracy', n_jobs=-1, cv=cv, random_state=seed)

# Executa busca
resultado = busca.fit(X_train, y_train)

# Resume resultados
print('Resultados busca - IQC Alterado')
print('Melhor acurácia: %s' % resultado.best_score_)
print('Melhor hiperparâmetro: %s' % resultado.best_params_)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Resultados busca - IQC Alterado
Melhor acurácia: 0.875
Melhor hiperparâmetro: {'estimator__dic_training_params': {'max_iter': 1000, 'accuracy_succ': 0.99, 'plot_graphs_and_metrics': False, 'plot_graphs_in_classifier': False, 'random_seed': 1, 'learning_rate': 0.01, 'do_classes_refit': True, 'reset_weights_epoch': 0, 'batch': 1, 'coupling_constants': [1]}, 'estimator__dic_classifier_params': {'sigma_q_params': [0.2, 0.9, 0.8, 0.7000000000000001], 'use_polar_coordinates_on_sigma_q': False, 'load_inputvector_env_state': False, 'normalize_axis': 0}, 'estimator__classifier_function': <function iqc_classifier at 0x000002123EE24EE0>}
CPU times: total: 2.03 s
Wall time: 10h 51min 57s


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  1.3min finished


## Search for best params on sigmaQ = rx\*sigmaX + ry\*sigmaY + rz\*sigmaZ

In [16]:
dic_training_params = {"max_iter": 1000,
    "accuracy_succ": 0.99,
    "plot_graphs_and_metrics": False,
    "plot_graphs_in_classifier": False,
    "random_seed": 1,
    "learning_rate": 0.01,
    "do_classes_refit":True,
    "reset_weights_epoch":0,
    "batch":1}

In [17]:
dic_classifier_params = {}
dic_classifier_params["sigma_q_params"] = [1,1,1,0]
dic_classifier_params["use_polar_coordinates_on_sigma_q"] = True
dic_classifier_params["load_inputvector_env_state"] = load_inputvector_env_state
dic_classifier_params["normalize_axis"] = 0

In [18]:
icq = OneVsRestClassifier(IQCClassifier(
                        classifier_function=iqc_classifier, 
                        dic_classifier_params=dic_classifier_params,
                        dic_training_params=dic_training_params), n_jobs=-1, verbose=1)

In [22]:
params = dict()
params["estimator__dic_training_params"] = [dic_training_params]
params["estimator__classifier_function"] = [iqc_classifier]
params["estimator__dic_classifier_params"] = []
for i in np.arange(0, np.pi, 0.1):
    for j in np.arange(0, np.pi, 0.1):
        for k in np.arange(0, np.pi, 0.1):
                    dic_classifier_params = {}
                    dic_classifier_params["sigma_q_params"] = [i, j, k]
                    dic_classifier_params["use_polar_coordinates_on_sigma_q"] = True
                    dic_classifier_params["load_inputvector_env_state"] = load_inputvector_env_state
                    dic_classifier_params["normalize_axis"] = normalize_axis
                    params["estimator__dic_classifier_params"].append(dic_classifier_params)

In [23]:
%%time
busca = RandomizedSearchCV(icq, params, n_iter=100, scoring='accuracy', n_jobs=-1, cv=cv, random_state=seed, verbose=60)

# Executa busca
resultado = busca.fit(X_train, y_train)

# Resume resultados
print('Resultados busca - ICQ Alterado')
print('Melhor acurácia: %s' % resultado.best_score_)
print('Melhor hiperparâmetro: %s' % resultado.best_params_)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Resultados busca - ICQ Alterado
Melhor acurácia: 0.9083333333333334
Melhor hiperparâmetro: {'estimator__dic_training_params': {'max_iter': 1000, 'accuracy_succ': 0.99, 'plot_graphs_and_metrics': False, 'plot_graphs_in_classifier': False, 'random_seed': 1, 'learning_rate': 0.01, 'do_classes_refit': True, 'reset_weights_epoch': 0, 'batch': 1, 'coupling_constants': [1]}, 'estimator__dic_classifier_params': {'sigma_q_params': [0.6000000000000001, 2.9000000000000004, 1.8], 'use_polar_coordinates_on_sigma_q': True, 'load_inputvector_env_state': False, 'normalize_axis': 0}, 'estimator__classifier_function': <function iqc_classifier at 0x000002123EE24EE0>}
CPU times: total: 2.19 s
Wall time: 10h 27min 34s


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  1.3min finished
